In [1]:
import tensorflow as tf
import numpy as np

# Tensor

+ 张量可以被简单理解为多维数组
+ 在张量中并没有真正保存数字，它保存的是如何得到这些数字的计算过程

In [2]:
a = tf.constant([[1,2,3],[1,2,3],[1,2,3],[1,2,3]], name='a')
b = tf.constant([[3,4,5],[3,4,5],[3,4,5],[3,4,5]], name='b')

result = tf.add(a,b, name='add')
print(result)

Tensor("add:0", shape=(4, 3), dtype=int32)


In [3]:
result.get_shape()

TensorShape([Dimension(4), Dimension(3)])

+ 一个张量中主要保存了三个属性
    + 名字，name
    + 纬度，shape
    + 类型，type

**`Tensor("add:0", shape=(4, 3), dtype=int32)`**
+ 张量的第一个属性名字不仅是一个张量的唯一标识符，它同样给出了这个张量是如何计算出来的
+ 张量和计算图上节点所代表的计算结果是对应的
    + "node : src_output" 的形式
    + src_output 表示当前张量来自节点的第几个输出

    + type：每个张量都会有唯一类型
    + 如果不指定类型，Tensorflow 会给出默认的类型
        +比如不带小数点的数会被默认为 int32，带小数点的会默认为 float32

+ **`张量的作用`**
    + 对中间计算结果的引用，提高代码的可读性
    + 通过张量来存储中间结果，方便获取中间结果
    + **`Tensor.get_shape()`** 可以快速获取张量的纬度信息

**`Tensor & Variable`**
+ Tensorflow 的核心概念是 Tensor，所有的数据都是通过 Tensor 的形式来组织的
+ Variable 的声明函数 tf.Variable() 只是一个运算
+ 这个运算的输出结果就是一个张量
+ **`变量只是一种特殊的张量`**

-----------------

# Constant & Variable

**`tf.constant`**(value, dtype=None, shape=None, name='Const', verify_shape=False)
+ tf.constant_initializer

In [4]:
cost = tf.constant([1,2,3,4])
with tf.Session() as sess:
    print(sess.run(cost))

[1 2 3 4]


In [5]:
bias = tf.constant(0.0, dtype=tf.float32, shape=[1,8])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(bias))

[[0. 0. 0. 0. 0. 0. 0. 0.]]


----------------------------------

**`tf.Variable`**(initial_value, trainable, collections, validate_shape, name, dtype)
+ initial_value : A `Tensor`, or Python object convertible to a `Tensor`, which is the initial value for the Variable.
+ trainable: 
    + If `True`, the default, also adds the variable to the graph collection `GraphKeys.TRAINABLE_VARIABLES`. 
    + This collection is used as the default list of variables to use by the `Optimizer` classes.
+ collections: 
    + List of graph collections keys. 
    + The new variable is added to these collections. Defaults to `[GraphKeys.GLOBAL_VARIABLES]`.
+ validate_shape: 
    + If `False`, allows the variable to be initialized with a value of unknown shape.
    + If `True`, the default, the shape of `initial_value` must be known.

In [6]:
zeros = tf.zeros([3,4])
var = tf.Variable(zeros)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(var))

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [7]:
weights = tf.Variable(tf.random_normal([2,3], stddev=2))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(weights))

[[ 3.262013   -2.954741   -0.5825089 ]
 [ 4.5530753  -1.4434229  -0.81389755]]


-------

# Variable & Placeholder & Fetch

**`Fetch`**
+ 在一个绘画中可以`同时执行多个op`

In [8]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)

add = tf.add(input2, input3)
mul = tf.multiply(input1, add)

with tf.Session() as sess:
    result = sess.run([mul, add])  # <<<　Fetch
    print("[mul, add] :", result)

[mul, add] : [21.0, 7.0]


----------------------------------

**`Feed`**
+ **在运行时, 才传入相应的值**
+ 占位符 : tf.placeholder(tf.float32)
+ feed_dict = {.. value ..}

In [9]:
input1 = tf.placeholder(tf.float32)  # 创建 float32 类型的占位符
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1, input2)

with tf.Session() as sess:
    res = sess.run(output, feed_dict={
        input1:[7.0],
        input2:[2.]
    })
    print(res)

[14.]


In [10]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)

[[-0.8403914  -0.45584244 -0.69373864]]


**`tf.placeholder() & feed_dict=`**
+ Tensorflow 提供了一个 placeholder 机制用于提供输入数据
+ placeholder 相当于定义一个位置，这个位置中的数据在程序运行时再指定


+ `tf.placeholder(tf.float32, shape=(2,3), name="input")`
    + 在 placeholder 定义时，这个位置上的数据类型是需要指定的，并不可以改变
    + placeholder 中的数据纬度信息，可以根据提供的数据推导出来，所以不一定要给出
    + 计算向前传播结果时，需要提供一个 feed_dict 来指定 x 的取值
    + feed_dict 是一个字典，在字典中需要给出每个用到的 placeholder 的取值

# get_variable & variable_scope("", reuse= )

**`tf.get_variable`**(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None)
+ 初始化更方便
```
W = tf.get_variable("W", shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())
```

**`tf.variable_scope`**(name_or_scope, reuse=None)

+ 当神经网络的结构更加复杂, 参数更多时, 就需要一个更好的方式来传递和管理神经网络中的参数
+ Tensorflow 提供了`通过变量名称`来创建或者获取一个变量的机制
+ 通过这个机制, 在不同的函数中可以直接通过变量的名字来使用变量, 而不需要将变量通过参数的形式到处传递

+ tf.get_variable() 用于创建变量时, 它和 tf.Variable() 的功能是基本等价的
    + 对于 tf.Variable 函数, 变量名称是一个可选的参数
    + 对于 tf.get_variable 函数, 变量名称是一个必选参数
```
    v = tf.get_variable("var_00", shape=[1], initializer=tf.constant_initializer(1.0))
    v = tf.Variable(tf.constant(1.0, shape=[1]), name='var_00')
```

----

**`tf.get_variable 工作机制`**

+ 当 `tf.get_variable_scope().reuse == False`，调用该函数会创建新的变量
```
      with tf.variable_scope("foo"):
          v = tf.get_variable("v", [1])
      assert v.name == "foo/v:0"
```
    　　

+ 当 `tf.get_variable_scope().reuse == True`，调用该函数会重用已经创建的变量
```
      with tf.variable_scope("foo"):
          v = tf.get_variable("v", [1])
      with tf.variable_scope("foo", reuse=True):
          v1 = tf.get_variable("v", [1])
      assert v1 is v
```

----

**`tf.variable_scope 工作机制`**

+ `tf.variable_scope()`用来指定变量的作用域，作为变量名的前缀，支持嵌套
```
    with tf.variable_scope("foo"):
        with tf.variable_scope("bar"):
            v = tf.get_variable("v", [1])
    assert v.name == "foo/bar/v:0"
```

+ 当前环境的作用域可以通过函数 `tf.get_variable_scope()` 获取
+ 并且 **`reuse flag`** 可以通过调用 `reuse_variables()` 设置为True
```
    with tf.variable_scope("foo"):
        v = tf.get_variable("v", [1])
        tf.get_variable_scope().reuse_variables()
        v1 = tf.get_variable("v", [1])
    assert v1 is v
```

----

**`Sharing variables`**
+ 使用`tf.variable_scope()`来生成一个上下文管理器, 在这个上下文管理器中, 通过`tf.get_variable()`来`创建/获取`变量
+ **`tf.variable_scope(., reuse=True)`** & **`tf.get_variable()`** 获取
    + 获取 : 如果变量不存在, 则tf.get_variable()将报错
    + 创建 : 如果同名的变量已经存在, 则tf.get_variable()函数将报错

**`reuse = tf.AUTO_REUSE`**
+ 自动复用
+ 如果变量存在则复用，不存在则创建
+ 避免了代码的繁琐

In [11]:
def foo():
    with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1], initializer=tf.ones_initializer)
    return v

v1 = foo()  # Creates v.
v2 = foo()  # Gets the same, existing v.

print(v1.name)
print(v2.name)
assert v1 == v2

foo/v:0
foo/v:0


**`Example - 00`**

In [12]:
with tf.Session() as sess:
    with tf.variable_scope("Tyrion"):
        var = tf.get_variable('var_01', [1], initializer=tf.zeros_initializer)
        print(var.name)

    with tf.variable_scope("Tyrion", reuse=True):
        get_var = tf.get_variable("var_01")
        tf.global_variables_initializer().run()
        print(sess.run(get_var))

Tyrion/var_01:0
[0.]


**`Example - 01`**

In [13]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("var_02", [1])
        assert v.name == "foo/bar/var_02:0"

**`Example - 02`**

In [14]:
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [15]:
# 需要两个卷积层，可以通过 tf.variable_scope() 指定作用域进行区分
# 如 with tf.variable_scope("conv1") 指定了第一个卷积层作用域为conv1
# 在这个作用域下有两个变量 weights 和 biases

def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

+ 在 image_filters 这个作用域, 重复使用第一张图片输入时创建的变量，调用函数reuse_variables()
```
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(image1)
    scope.reuse_variables()
    result2 = my_image_filter(image2)
```

**`Example - 03`**

In [16]:
def func(in_put, in_channel, out_channel):
    with tf.variable_scope(name_or_scope='', reuse=tf.AUTO_REUSE): ### 改动部分 ###
        weights = tf.get_variable(name="weights", shape=[2, 2, in_channel, out_channel], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        convolution = tf.nn.conv2d(input=in_put, filter=weights, strides=[1, 1, 1, 1], padding="SAME")
        return convolution

def main():
    with tf.Graph().as_default():
        input_x = tf.placeholder(dtype=tf.float32, shape=[1, 4, 4, 1])
        for _ in range(5):    # 重复调用
            output = func(input_x, 1, 1)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                _output = sess.run([output], feed_dict={
                    input_x : np.random.uniform(low=0, high=255, size=[1, 4, 4, 1])
                })
                # print(_output)

if __name__ == "__main__":
    main()

--------------------------------------------------------

# generator

+ **`Use already have value`**
+ **`Fix Value`**
+ **`Sequence`**
+ **`Random Value`**
+ **`initializer`**

**`Tensor.initialized_value()`**

In [17]:
weights = tf.Variable(tf.random_normal([2,3], stddev=2))

weights_ = tf.Variable(weights.initialized_value() * 2.0)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(weights))
    print()
    print(sess.run(weights_))

[[-1.2913926 -0.9373361  2.606292 ]
 [-1.7561935 -0.8057926 -1.7381574]]

[[-2.5827851 -1.8746722  5.212584 ]
 [-3.512387  -1.6115853 -3.4763148]]


**`Fix Value`**
+ tf.`fill`(dims, value, name=None)
+ tf.`zeros`(shape, dtype=tf.float32, name=None) / tf.`ones`( )
+ tf.`zeros_like`(tensor, dtype=None, name=None, optimize=True) / tf.`ones_like`( )

In [18]:
f_one = tf.fill([3,4], 1)

with tf.Session() as sess:
    print(sess.run(f_one))

[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]


In [19]:
ones = tf.ones([4,4])
zeros = tf.zeros([3,4])
zeros_l = tf.zeros_like(ones)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(zeros))
    print(sess.run(zeros_l))
    print()
    print(zeros.get_shape())
    print(zeros_l.get_shape())

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

(3, 4)
(4, 4)


**`Sequence`**

+ **`tf.linspace(start, stop, num, name=None)`**
    + 在 [start,stop] 范围内产生 num 个数的等差数列
    + start & stop 要用浮点数表示
    
+ **`tf.range(start, limit=None, delta=1, dtype=None, name='range')`**
    + 在 [start,limit) 范围内以步进值 delta 产生等差数列

In [20]:
x = tf.linspace(start=1.0, stop=5.0, num=5, name=None)
y = tf.range(start=1,limit=5, delta=1)
with tf.Session() as sess:
    print(sess.run(x))
    print(sess.run(y))

[1. 2. 3. 4. 5.]
[1 2 3 4]


**`Random Value`**

+ **`tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a normal distribution.
    + 正态分布

+ **`tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a truncated normal distribution.
    + 截断正态分布，只保留 [mean-2*stddev, mean+2*stddev] 内的随机数

+ **`tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a uniform distribution.
    + 均匀分布 : [minval, maxval)

**`initializer`**

+ tf.ones_initializer(dtype=tf.float32)
+ tf.zeros_initializer(dtype=tf.float32)
+ tf.constant_initializer(value=0, dtype=tf.float32, verify_shape=False)
+ tf.random_uniform_initializer(minval=0, maxval=None, seed=None, dtype=tf.float32)
+ tf.random_normal_initializer(mean=0.0, stddev=1.0, seed=None, dtype=tf.float32)
+ tf.truncated_normal_initializer(mean=0.0, stddev=1.0, seed=None, dtype=tf.float32)

---------------

# Graph & Session

+ Tensorflow 中的每一个计算都是计算图上的一个节点，而节点之间的边描述了计算之间的依赖关系


+ Tensorflow 程序中，系统会自动维护一个默认的计算图
    + **`tf.get_default_graph()`**：获取当前默认的计算图
    + **`Tensor.graph`**：查看张量所属的计算图

In [21]:
a = tf.constant(1)
a.graph is tf.get_default_graph()

True

+ **`tf.Graph()`**:生成新的计算图
    + `with tf.Graph().as_default()`
    + `with tf.Session(graph=…) as sess:`


+ **`不同计算图上的 "张量 & 运算" 都不会共享`**

In [22]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('var_03', initializer=tf.zeros(shape=[1]))

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('var_03', initializer=tf.ones(shape=[1]))

with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('var_03')), "in Graph-01")

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('var_03')), "in Graph-02")

[0.] in Graph-01
[1.] in Graph-02


----------

**`Session / 会话`**

+ `会话拥有并管理 Tensorflow 程序运行时的所有资源`


+ 当所有计算完成之后，需要关闭会话来帮助系统回收资源，否则就可能出现资源泄露的问题
    + Method.01 : 明确调用会话生成和关闭函数
    + Method.02 : 为了解决异常退出时，资源释放的问题
        + Tensorflow 可以通过 Python 的上下文管理器来使用会话
        + 当上下文管理器退出时，则会自动释放所有资源

`Method-01`

In [23]:
sess = tf.Session()
# sess.run(...)
sess.close()

`Method-02`

In [24]:
with tf.Session() as sess:
    pass

+ 与 graph 不同的是, **`Tensorflow 不会自动生成默认的会话，而是需要手动指定`**
    + `sess.as_default`

In [25]:
test = tf.constant([1,2,3], name='test')

sess = tf.Session()
with sess.as_default():
    print(sess is tf.get_default_session())
    
    print(test.eval() == sess.run(test))
    print(test.eval())
    print(sess.run(test))

True
[ True  True  True]
[1 2 3]
[1 2 3]


**`Tensor.eval() & tf.get_default_session().run(Tensor)`**
+ **`sess.run() 可以一步获取多个tensor中的值`**

In [26]:
a = tf.constant([1, 2, 3], name='a')
b = tf.constant([3, 4, 5], name='b')

add = tf.add(a, b, name='add')
mul = tf.multiply(a, b, name='mul')

with tf.Session() as sess:
    print(sess.run([add, mul]))
    print(add.eval(session=sess), mul.eval(session=sess))

[array([4, 6, 8]), array([ 3,  8, 15])]
[4 6 8] [ 3  8 15]


-------

# tf.collection / 集合 / 整合资源

+ 在一个计算图中，可以通过集合 collection 来管理不同类别的资源
    + 资源类型：张量、变量

    
+ 通过 `tf.add_to_collection()` 可以将资源加入一个或多个集合中
+ 然后通过 `tf.get_collection()` 获取一个集合里面的所有资源

**`tf.GraphKeys`**

+ tf.GraphKeys.**GLOBAL_VARIABLES**
    + 默认的 Variable 对象集合，在分布式环境共性
    + 使用场景：持久化 Tensorflow 模型


+ tf.GraphKeys.**LOCAL_VARIABLES**
    + 每台计算机的局部变量
    + 通常用于临时变量，如：计数器
    + 使用 tf.contrib.framework.local_variable 添加到此集合


+ tf.GraphKeys.**MODEL_VARIABLES**
    + 在构建模型时，所有用于正向传播的 Variable 都将添加到这里
    + 使用 tf.contrib.framework.model_variable 添加到此集合


+ tf.GraphKeys.**TRAINABLE_VARIABLES**
    + 将有优化器训练的变量
    + tf.trainable_variables()


+ tf.GraphKeys.**SUMMARIES**
    + 日志生成相关的张量
    + Tensorflow 计算可视化


+ tf.GraphKeys.**MOVING_AVERAGE_VARIABLES**
    + 所有计算了滑动平均值的变量
    + tf.moving_average_variables


+ tf.GraphKeys.**REGULARIZATION_LOSSES**
    + 收集在图构造期间正则化损失

**`tf.GraphKeys 使用`**

----

# operation / op

**`Property`**
+ `Tensor.get_shape()`
+ `tf.shape(tensor)` || sess.run()
+ `tf.reshape(tensor, shape, name=None)`

In [30]:
tensor = tf.Variable(tf.ones([3,4], dtype=tf.float32))
print(tensor.get_shape())

(3, 4)


In [34]:
# tf.shape() 函数本身也是返回一个张量，因而，需要用 sess.run() 来得到具体的值
tensor = tf.Variable(tf.ones([3,4], dtype=tf.float32))

with tf.Session() as sess:
    print(sess.run(tf.shape(tensor)))

[3 4]


In [35]:
tensor = tf.Variable(tf.ones([3,4], dtype=tf.float32))
temp = tf.reshape(tensor, [-1,1])
print(temp.get_shape())

(12, 1)


In [ ]:
**`Basic Math Method`**

In [ ]:
a = tf.constant([[1,2,3],[1,2,3],[1,2,3],[1,2,3]], name='a')
b = tf.constant([[3,4,5],[3,4,5],[3,4,5],[3,4,5]], name='b')

result = tf.add(a,b, name='add')
print(result)

In [ ]:
assign

In [ ]:
**`Implement Loop add one`**

In [ ]:
count = tf.Variable(0, name='counter')
one = tf.constant(1)
new_value = tf.add(count, one)

In [ ]:
**赋值op : tf.assign(old, new)**

In [ ]:
update = tf.assign(count, new_value)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(count))
    for _ in range(5):
        sess.run(update)
        print(sess.run(count))

In [ ]:
product = tf.matmul(m1, m2)

In [ ]:
print(product)

In [ ]:
**`Statistics Method`**

In [ ]:
calc on some axis

In [ ]:
input_tensor:输入
    axis：在哪个维度进行 .. 操作
    keepdims:是否保留原始数据的唯独，False 相当于执行完后，原始数据就会少一个维度
        
        

In [ ]:
all
row
column

In [ ]:
tf.reduce_all()
计算tensor中各个元素的逻辑和（and运算）

In [ ]:
tf.reduce_any
计算tensor中各个元素的逻辑或（or运算）

In [ ]:
tf.reduce_join

In [ ]:
tf.reduce_max

In [ ]:
tf.reduce_mean(input_tensor, axis=None, keepdims=None, name=None, reduction_indices=None, keep_dims=None)
Computes the mean of elements across dimensions of a tensor.
Reduces `input_tensor` along the dimensions given in `axis`.
Unless `keepdims` is true, the rank of the tensor is reduced by 1 for each
entry in `axis`. If `keepdims` is true, the reduced dimensions
are retained with length 1.

If `axis` is None, all dimensions are reduced, and a
tensor with a single element is returned.



In [47]:
x = tf.constant([[1., 1., 1.], [2., 2., 2.]])
tf.reduce_mean(x)  # 1.5
tf.reduce_mean(x, 0)  # [1.5, 1.5, 1.5]
tf.reduce_mean(x, 1)  # [1., 2.]
tf.reduce_mean(x, 1, keepdims=True)  # [[1.], [2.]] / (2, 1)

<tf.Tensor 'Mean_12:0' shape=(2, 1) dtype=float32>

In [ ]:
tf.reduce_min

In [ ]:
tf.reduce_prod

**`tf.reduce_sum`**(input_tensor, axis=None, keepdims=None, name=None, reduction_indices=None, keep_dims=None)
+ Computes the sum of elements across dimensions of a tensor.
+ keepdims: If true, retains reduced dimensions with length 1.

In [39]:
x = tf.constant([[1, 1, 1], [1, 1, 1]])
tf.reduce_sum(x)  # 6
tf.reduce_sum(x, 0)  # [2, 2, 2]
tf.reduce_sum(x, 1)  # [3, 3]
tf.reduce_sum(x, 1, keepdims=True)  # [[3], [3]]  tf.reduce_sum(x, 1, keepdims=True).get_shape()
tf.reduce_sum(x, [0, 1])  # 6

<tf.Tensor 'Sum_4:0' shape=() dtype=int32>

**`Assemble / Split data`**

**`tf.concat(values, axis, name='concat')`**
+ Concatenates tensors along one dimension.

In [38]:
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]

t3 = tf.concat([t1, t2], 0)
t4 = tf.concat([t1, t2], 1)

with tf.Session() as sess:
    print(sess.run(t3))
    print()
    print(sess.run(t4))

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]]


In [ ]:
+ **`tf.random_shuffle()`**
tf.one_hot